## Libraries

In [2]:
import os
import yaml
import numpy as np
import math
from glob import glob

import warnings

warnings.filterwarnings("ignore")

# Local Module Imports
import sys
import importlib

sys.path.insert(0, "../src")

# import polygon_handle

# importlib.reload(polygon_handle)
# from polygon_handle import masks_to_polygons

import log_setup

importlib.reload(log_setup)
from log_setup import logger

import eval

importlib.reload(eval)
from eval import (
    hausdorff_dist_wkt,
    iou_wkt,
    strided_temporal_consistency,
    save_results_to_csv,
)

import visualization

importlib.reload(visualization)
from visualization import create_boxplot

import utils

importlib.reload(utils)
from utils import get_files_dirs_ext

## Data Sources

In [3]:
# Base directory
current_dir = os.getcwd()
BASE_DIR = os.path.dirname(current_dir)

# Load 'config' file
config_file = os.path.join(BASE_DIR, "config.yml")
with open(config_file, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# Output directory
output_dir = "../outputs"

# Training set directory
train_dir = os.path.join(BASE_DIR, config["data"]["full"]["train_dir"])

# Testing set wkt file path
test_wkt_path = os.path.join(BASE_DIR, config["data"]["wkt"]["test_wkt"])

# Pyspatiotemporalgeom, Shape Based, CVAE output directories
pstg_dir = os.path.join(output_dir, "pstg")
shpe_dir = os.path.join(output_dir, "shape")
cvae_dir = os.path.join(output_dir, "CVAE")

### PNG to WKT conversion

In [5]:


wkt_files = get_files_dirs_ext([pstg_dir, shpe_dir, cvae_dir], "wkt", return_paths=True)
png_dirs = get_files_dirs_ext([pstg_dir, shpe_dir, cvae_dir], "png")
# show wkt files and png dirs on a pretty table with several rows

In [6]:
for wkt_file in sorted(wkt_files):
    # print like a table
    print(f"{wkt_file}")
print("Total number of WKTs: ", len(wkt_files))
print()

for png_dir in sorted(png_dirs):
    # print like a table
    print(f"{png_dir}")
print("Total number of PNG dirs: ", len(png_dirs))

../outputs/CVAE/extrapol/70/full/WKT/file.wkt
../outputs/CVAE/extrapol/70/sampled/WKT/file.wkt
../outputs/CVAE/extrapol/70/unet/WKT/extrapol_unet.wkt
../outputs/CVAE/extrapol/90/full/WKT/file.wkt
../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
../outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt
../outputs/CVAE/interpol/full/WKT/file.wkt
../outputs/CVAE/interpol/sampled/WKT/file.wkt
../outputs/pstg/interpol/full/WKT/pyspatial_interpol.wkt
../outputs/pstg/interpol/sampled/WKT/pyspatial_interpol_sampled.wkt
../outputs/shape/interpol/full/WKT/shape_interpol.wkt
../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt
Total number of WKTs:  12

../outputs/CVAE/extrapol/70/full/PNG
../outputs/CVAE/extrapol/70/sampled/PNG
../outputs/CVAE/extrapol/70/unet/PNG
../outputs/CVAE/extrapol/90/full/PNG
../outputs/CVAE/extrapol/90/sampled/PNG
../outputs/CVAE/extrapol/90/unet/PNG
../outputs/CVAE/interpol/full/PNG
../outputs/CVAE/interpol/sampled/PNG
Total number of PNG dirs:  8


## [BurnedAreaUAV](https://zenodo.org/records/7944963) Test Set Evalution

In [7]:
logger.info("Reading WKTs files...")
with open(os.path.join(BASE_DIR, config["data"]["wkt"]["train_wkt"]), "r") as f:
    train_wkt = f.read().splitlines()

with open(os.path.join(BASE_DIR, config["data"]["wkt"]["test_wkt"]), "r") as f:
    test_wkt = f.read().splitlines()

# evaluation set dictionary
eval_set_dict = {}
for i, wkt in enumerate(train_wkt):
    eval_set_dict[i*100] = wkt
for i, wkt in enumerate(test_wkt):
    eval_set_dict[(i*100)+20250] = wkt
eval_set_dict = dict(sorted(eval_set_dict.items()))

INFO - Reading WKTs files...


### Hausdorf Distance

In [8]:
# Create the pattern to match the WKT files
pattern_interpol = os.path.join(output_dir, "*", "interpol", "*", "WKT", "*.wkt")
interpol_wkt_files = glob(pattern_interpol)

# Create the pattern to match the WKT files
pattern_extrapol_70 = os.path.join(output_dir, "*", "extrapol", "70", "*", "WKT", "*.wkt")
extrapol_wkt_files_70 = glob(pattern_extrapol_70)

# Create the pattern to match the WKT files
pattern_extrapol_90 = os.path.join(output_dir, "*", "extrapol", "90", "*", "WKT", "*.wkt")
extrapol_wkt_files_90 = glob(pattern_extrapol_90)

In [9]:
interpol_wkt_files

['../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt',
 '../outputs/shape/interpol/full/WKT/shape_interpol.wkt',
 '../outputs/CVAE/interpol/sampled/WKT/file.wkt',
 '../outputs/CVAE/interpol/full/WKT/file.wkt',
 '../outputs/pstg/interpol/sampled/WKT/pyspatial_interpol_sampled.wkt',
 '../outputs/pstg/interpol/full/WKT/pyspatial_interpol.wkt']

In [10]:
train_files_num = len(glob(os.path.join(train_dir, "*", "*.png")))
frame_70_perc = math.ceil(train_files_num * (70))
final_70_frame = min(eval_set_dict.keys(), key=lambda x:abs(x-frame_70_perc))

frame_90_perc = math.ceil(train_files_num * (90))
final_90_frame = min(eval_set_dict.keys(), key=lambda x:abs(x-frame_90_perc))
print(f"70%: {final_70_frame}, 90%: {final_90_frame}")

70%: 15800, 90%: 20350


In [11]:
# from eval set dict, choose all th keys >= frame_70_perc
eval_set_dict_70 = {k: v for k, v in eval_set_dict.items() if k >= final_70_frame}
eval_set_70 = list(eval_set_dict_70.keys())
# save the sorted dict to a WKT file
with open(os.path.join(BASE_DIR, "eval_70.wkt"), "w") as f:
    for key, poly in eval_set_dict_70.items():
        f.write(str(poly))
        f.write("\n")

# from eval set dict, choose all th keys >= frame_70_perc
eval_set_dict_90 = {k: v for k, v in eval_set_dict.items() if k >= final_90_frame}
eval_set_90 = list(eval_set_dict_90.keys())
# save the sorted dict to a WKT file
with open(os.path.join(BASE_DIR, "eval_90.wkt"), "w") as f:
    for key, poly in eval_set_dict_70.items():
        f.write(str(poly))
        f.write("\n")

In [10]:
for wkt_file in interpol_wkt_files:
 
    # test set frame indexes 
    idx = np.linspace(20250, 22450, 23).astype(int)
    mean_hd, hd_list = hausdorff_dist_wkt(gt_wkt=test_wkt_path, model_wkt= wkt_file, eval_idx=idx)

    logger.info(f"Hausdorff Distance for \033[1m{wkt_file}\033[0m")
    logger.info(f"Mean: {mean_hd:10.4f}, Std Dev.: {np.std(hd_list):10.4f}, Min: {np.min(hd_list):10.4f}, Max: {np.max(hd_list):10.4f}")
    logger.info("...........................................................................\n") 

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt
INFO - Mean:   108.3490, Std Dev.:     8.6310, Min:    91.9837, Max:   125.5388
INFO - 

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/shape/interpol/full/WKT/shape_interpol.wkt
INFO - Mean:    60.8145, Std Dev.:    33.3117, Min:    19.4440, Max:   117.0000
INFO - 

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/interpol/sampled/WKT/file.wkt
INFO - Mean:    87.7788, Std Dev.:     9.1669, Min:    76.5376, Max:   119.5492
INFO - 

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/interpol/full/WKT/file.wkt
INFO - Mean:    50.3446, Std Dev.:    28.8379, Min:    20.8614, Max:   113.2652
INFO - 

INFO - Reading WKTs files...
INFO - 

In [12]:
eval70_wkt = os.path.join(BASE_DIR, "eval_70.wkt")

for wkt_file in extrapol_wkt_files_70:
    mean_hd, hd_list = hausdorff_dist_wkt(
        gt_wkt=eval70_wkt,
        model_wkt=wkt_file,
        eval_idx=eval_set_70[:-1],
    )

    logger.info(f"Hausdorff Distance for \033[1m{wkt_file}\033[0m")
    logger.info(
        f"Mean: {mean_hd:10.4f}, Std Dev.: {np.std(hd_list):10.4f}, Min: {np.min(hd_list):10.4f}, Max: {np.max(hd_list):10.4f}"
    )
    logger.info("\n")

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/70/sampled/WKT/file.wkt
INFO - Mean:   190.7728, Std Dev.:    28.3400, Min:   131.8560, Max:   221.9031
INFO - 

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/70/full/WKT/file.wkt
INFO - Mean:   105.8319, Std Dev.:    33.9872, Min:    30.0832, Max:   148.7616
INFO - 

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/70/unet/WKT/extrapol_unet.wkt
INFO - Mean:   111.1561, Std Dev.:    35.4082, Min:    40.3113, Max:   153.6880
INFO - 



In [16]:
eval90_wkt = os.path.join(BASE_DIR, "eval_90.wkt")

for wkt_file in extrapol_wkt_files_90:
    mean_hd, hd_list = hausdorff_dist_wkt(
        gt_wkt=eval90_wkt,
        model_wkt=wkt_file,
        eval_idx=eval_set_90[:-1],
    )

    logger.info(f"Hausdorff Distance for \033[1m{wkt_file}\033[0m")
    logger.info(
        f"Mean: {mean_hd:10.4f}, Std Dev.: {np.std(hd_list):10.4f}, Min: {np.min(hd_list):10.4f}, Max: {np.max(hd_list):10.4f}"
    )
    logger.info("\n")

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
INFO - Mean:    95.0263, Std Dev.:    18.8012, Min:    72.3395, Max:   130.0000
INFO - 

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/full/WKT/file.wkt
INFO - Mean:   102.9551, Std Dev.:    31.4315, Min:    51.0000, Max:   142.8881
INFO - 

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt
INFO - Mean:    96.6925, Std Dev.:    30.0752, Min:    46.2709, Max:   139.5421
INFO - 



In [70]:
for wkt_file in extrapol_wkt_files_90:
    mean_hd, hd_list = hausdorff_dist_wkt(test_wkt_path, wkt_file)

    logger.info(f"Hausdorff Distance for \033[1m{wkt_file}\033[0m")
    logger.info(f"Mean: {mean_hd:10.4f}, Std Dev.: {np.std(hd_list):10.4f}, Min: {np.min(hd_list):10.4f}, Max: {np.max(hd_list):10.4f}")
    logger.info("\n") 

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
INFO - Mean:   457.0036, Std Dev.:     2.1096, Min:   452.2168, Max:   460.8959
INFO - 

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/full/WKT/file.wkt
INFO - Mean:   559.4281, Std Dev.:     1.6240, Min:   556.6803, Max:   562.3131
INFO - 

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt
INFO - Mean:   565.7227, Std Dev.:     1.8018, Min:   561.5140, Max:   568.9895
INFO - 



In [ ]:
# Store data in a dictionary
data = {
    "Shape Based": {"full": hd_list_shp_ts, "sampled": hd_list_shp_pr},
    "CVAE": {"full": hd_list_cvae_ts, "sampled": hd_list_cvae_pr},
    "McKenney": {"full": hd_list_pstg_ts, "sampled": hd_list_pstg_pr},
}

# Positions for bars
positions = {"full": [1, 1.5, 2], "sampled": [2.75, 3.25, 3.75]}

# Colors for boxes
colors = ["lightgreen", "lightblue", "lightcoral"]

# Create boxplot
create_boxplot(data, positions, colors)

### IoU 

In [11]:
for wkt_file in interpol_wkt_files:
 
    # test set frame indexes 
    idx = np.linspace(20250, 22450, 23).astype(int)
    mean_iou, iou_list = iou_wkt(gt_wkt=test_wkt_path, model_wkt= wkt_file, eval_idx=idx)

    logger.info(f"IoU for \033[1m{wkt_file}\033[0m")
    logger.info(f"Mean: {mean_iou:10.4f}, Std Dev.: {np.std(iou_list):10.4f}, Min: {np.min(iou_list):10.4f}, Max: {np.max(iou_list):10.4f}")
    logger.info("...........................................................................\n")
    

INFO - Reading WKTs files...
INFO - Calculating IoU...
INFO - IoU for ../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt
INFO - Mean:     0.9296, Std Dev.:     0.0210, Min:     0.8867, Max:     0.9640
INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Calculating IoU...
INFO - IoU for ../outputs/shape/interpol/full/WKT/shape_interpol.wkt
INFO - Mean:     0.9589, Std Dev.:     0.0155, Min:     0.9253, Max:     0.9767
INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/interpol/sampled/WKT/file.wkt
INFO - Mean:     0.9083, Std Dev.:     0.0166, Min:     0.8685, Max:     0.9345
INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/interpol/full/WKT/file.wkt
INFO - Mean:     0.9

In [14]:
for wkt_file in extrapol_wkt_files_70:
    mean_iou, iou_list = iou_wkt(
        gt_wkt=eval70_wkt,
        model_wkt=wkt_file,
        eval_idx=eval_set_70[:-1],
    )

    logger.info(f"IoU for \033[1m{wkt_file}\033[0m")
    logger.info(
        f"Mean: {mean_iou:10.4f}, Std Dev.: {np.std(iou_list):10.4f}, Min: {np.min(iou_list):10.4f}, Max: {np.max(iou_list):10.4f}"
    )
    logger.info("...........................................................................\n")

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/70/sampled/WKT/file.wkt
INFO - Mean:     0.6467, Std Dev.:     0.0370, Min:     0.5987, Max:     0.7244
INFO - 

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/70/full/WKT/file.wkt
INFO - Mean:     0.8369, Std Dev.:     0.0488, Min:     0.7754, Max:     0.9487
INFO - 

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/70/unet/WKT/extrapol_unet.wkt
INFO - Mean:     0.8249, Std Dev.:     0.0489, Min:     0.7583, Max:     0.9341
INFO - 



In [18]:
for wkt_file in extrapol_wkt_files_90:
    mean_iou, iou_list = iou_wkt(
        gt_wkt=eval90_wkt,
        model_wkt=wkt_file,
        eval_idx=eval_set_90[:-1],
    )

    logger.info(f"IoU for \033[1m{wkt_file}\033[0m")
    logger.info(
        f"Mean: {mean_iou:10.4f}, Std Dev.: {np.std(iou_list):10.4f}, Min: {np.min(iou_list):10.4f}, Max: {np.max(iou_list):10.4f}"
    )
    logger.info("...........................................................................\n")

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
INFO - Mean:     0.8697, Std Dev.:     0.0385, Min:     0.8119, Max:     0.9231
INFO - ...........................................................................

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/90/full/WKT/file.wkt
INFO - Mean:     0.8531, Std Dev.:     0.0425, Min:     0.7918, Max:     0.9328
INFO - ...........................................................................

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt
INFO - Mean:     0.8621, Std Dev.:     0.0341, Min:     0.8027, Max:     0.

## Temporal Consistency

In [12]:
strides = [1, 10, 100, 1000, 10000]
# calculate temporal consistency for each model in each mode
for wkt_file in wkt_files:
    tc = strided_temporal_consistency(
        wkt_file=wkt_file,
        num_polygons=22500,
        strides=strides
    )
    save_path = os.path.join(os.path.dirname(wkt_file), "tc")
    save_results_to_csv(results=tc, base_filename=save_path)

DEBUG - Reading WKT file...
DEBUG - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...
INFO - Calculating the temporal consistency with stride 10...
INFO - Calculating the temporal consistency with stride 100...
INFO - Calculating the temporal consistency with stride 1000...
INFO - Calculating the temporal consistency with stride 10000...
DEBUG - Reading WKT file...
DEBUG - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...
INFO - Calculating the temporal consistency with stride 10...
INFO - Calculating the temporal consistency with stride 100...
INFO - Calculating the temporal consistency with stride 1000...
INFO - Calculating the temporal consistency with stride 10000...
DEBUG - Reading WKT file...
DEBUG - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...
INFO - Calculating the temporal consistency with stride 10...
INFO - Calculating the temporal consistency with stride 100...
INFO - 